In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [7]:
def count_missing(fullfilename):
    df = pd.read_csv(fullfilename)
    print(df.isna().sum())

In [ ]:
def create_map(fullfilename):
    df = pd.read_csv(fullfilename)
    

In [71]:
filepath = "ncbi\\aka_scraped"

In [49]:
def get_dups(fullfilepath):    
    df = pd.read_csv(fullfilepath)

    cols = ['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID', 'aka']

    present_map = {}
    dup_indices = []
    for index, row in df[cols][:-2].iterrows():
        curentry = tuple(row[['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID',]])
        present_map[curentry] = (False, [])

    for index, row in df[cols][:-2].iterrows():
        curentry = tuple(row[['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID',]])
        
        if present_map[curentry][0] == True:
            # dup_indices.append((index, present_map[curentry][1]))
            present_map[curentry][1].append(index)
        else:
            akas = str(list(row[['aka']])[0]).strip().split(';')

            for aka in akas:
                tmplist = list(curentry)
                tmplist[0] = aka
                akaentry = tuple(tmplist)

                if akaentry in present_map.keys() and present_map[akaentry][0] == True:
                    # dup_indices.append((index, present_map[curentry][1]))
                    present_map[akaentry][1].append(index)

            present_map[curentry] = (True, [index])

    return {fullfilepath: present_map}

In [68]:
delete_rows = {}

for filename in os.listdir(filepath):
    if(filename[-3:]=='csv'):
        fullfilename = os.path.join(filepath, filename)
        dups_data = get_dups(fullfilename)

        removerows = []

        for i in dups_data.values():
            for j in i:
                if len(i[j][1]) > 1:
                    for rownum in i[j][1][1:]:
                        removerows.append(rownum)

        delete_rows[filename] = removerows

import json
with open('final_dups_to_remove.json', 'w') as fp:
    json.dump(delete_rows, fp)

# remove row number+2 wali row in excel from the sheet

In [76]:
for filename in os.listdir(filepath):
    if(filename[-3:]=='csv'):
        fullfilename = os.path.join(filepath, filename)
        df = pd.read_csv(fullfilename)
        rows_to_delete = delete_rows[filename]
        if len(rows_to_delete)>0:
            df.drop(df.index[rows_to_delete], inplace=True)
            df.to_csv(os.path.join('ncbi\\aka_scraped\\dups_removed', 'dups_removed_'+filename), index=False)